In [67]:
print("ok")

ok


In [17]:
%pwd

'd:\\PROJECTS\\Gen AI Project\\Medical-Chatbot-Gen-AI\\research'

In [18]:
import os
os.chdir("../")

In [66]:
%pwd

'd:\\PROJECTS\\Gen AI Project\\Medical-Chatbot-Gen-AI'

In [68]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [69]:
#Extract data from the PDF File
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [70]:
extracted_data = load_pdf("Data/")


In [71]:
#extracted_data


In [72]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [74]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [75]:
from langchain.embeddings import HuggingFaceEmbeddings

In [76]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [77]:
embeddings = download_hugging_face_embeddings()

In [78]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [79]:
#query_result

In [80]:
from dotenv import load_dotenv
load_dotenv()

True

In [99]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [82]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medimate"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region= "us-east-1"
    )
)

In [98]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [84]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [85]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecode index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [86]:
docsearch

In [87]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [88]:
retrieved_docs = retriever.invoke("What is Acne")

In [89]:
retrieved_docs

[Document(id='affea901-a5e0-49bf-a8ed-3976bd16b590', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Medical_Book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='9a4e54c4-52a4-4345-a989-f49c34db16ef', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_Book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='ede01b8e-0cfe-4ac2-8ee2-0a3e3dc7db1c', metadata={'page': 38.0, 'page_label': '39', 'source': 'Data\\Medical_Book.pdf'}, page_conte

In [106]:
from langchain_openai import OpenAI
llm = OpenAI (temperature=0.4, max_tokens=500)

In [101]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system_prompt),
        ("human", "{input}"),

        
    ]
)


In [102]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [107]:
response = rag_chain.invoke({"input": "what is acne"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}